In [1]:
import time
import os
import win32com.client
import pymysql
import pandas as pd

class mySAP:
    def __init__(self, utente=None, password=None):
        self.apri_sap()
        self.session = self.connetti_sap()
        if utente:
            self.logon(utente, password)
            
        self.hostname = '37.156.244.12'
        self.username = 'emarchi1_admin'
        self.password = 'NGi&BcJt8!MB6MT4'
        self.database_name = 'emarchi1_lavoro'
        self.conn = pymysql.connect(host=self.hostname, user=self.username, password=self.password, db=self.database_name)
        
    def read_query(self, sql, args):
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(sql, args)
                result = cursor.fetchall()
                columns = [col[0] for col in cursor.description]
                df = pd.DataFrame(result, columns=columns)            
                return df
        finally:
            self.conn.close()    
        
    def apri_sap(self):
        # Avvia SAP GUI
        os.system("start saplogon.exe")
        time.sleep(5)  # Aspetta che SAP GUI si avvii
        
    def connetti_sap(self):
        # Connetti a SAP GUI tramite COM
        sapguiapp = win32com.client.GetObject("SAPGUI").GetScriptingEngine
        if sapguiapp.Children.Count > 0:
            connection = sapguiapp.Children(0)
        else:
            connection = sapguiapp.OpenConnection('INE [PUBLIC]')
        session = connection.Children(0)
        return session

    def logon(self, utente, password):
        self.session.findbyid('wnd[0]/usr/txtRSYST-BNAME').text = utente
        self.session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = password
        self.session.findById("wnd[0]").sendVKey(0)
        self.session.findById("wnd[0]").sendVKey(0)
        self.session.findById("wnd[0]").sendVKey(0)
        
mysap = mySAP('932197', '.Gioia8888')
mysap.session.StartTransaction('CN22')
